In [1]:
import pandas as pd
import random

In [2]:
def create_promoter_df_for_ml(positive_promoter_df,
                              negative_permuted_promoter_df,
                              random_df,
                              percentage_permuted,
                              random_idx_start):
    promoters_for_ML = pd.DataFrame()
    num_sequences_to_fill = list(range(0,positive_promoter_df.shape[0]))
    num_permuted_promoter = int(round(percentage_permuted * len(num_sequences_to_fill)))
    num_random_promoter = len(num_sequences_to_fill) - num_permuted_promoter
    permuted_promoters_to_pull_idx = random.sample(num_sequences_to_fill,num_permuted_promoter)

    negative_set_permuted_df = negative_permuted_promoter_df.copy().iloc[permuted_promoters_to_pull_idx,:]
    random_set_df = random_df.copy().iloc[0:num_random_promoter,:]

    random_idx_start = random_idx_start + num_random_promoter

    promoters_for_ML_df = pd.concat([promoters_for_ML,
           positive_promoter_df,
           negative_set_permuted_df,
           random_set_df],
            sort=False).reset_index().drop('index', axis=1)
    return(promoters_for_ML_df, random_idx_start)

In [3]:
percentage_permuted = 0.75 # convert to an argument
promoters_all_df = pd.read_csv('../../data/promoters_for_ML/promoters_all.csv', low_memory = False).fillna('')

EPDnew_df = promoters_all_df.copy()[(promoters_all_df['database/source'] == 'EPDnew') |
                                  (promoters_all_df['database/source'] == 'EPDnew (permuted)')
                                  ]

RegulonDB_df = promoters_all_df.copy()[(promoters_all_df['database/source'] == 'RegulonDB') |
                                  (promoters_all_df['database/source'] == 'RegulonDB (permuted)')
                                  ]

random_df = promoters_all_df.copy()[promoters_all_df['database/source'] == 'random']

eukaryotic_organisms = set(EPDnew_df['organism'])
eukaryotic_motifs = [ 'Inr-pfa', 'nonInr-pfa','TATA', 'nonTATA'] # set(EPDnew_df['sigma factor/motif'])

EPDnew_promoters_for_ML_df = pd.DataFrame()
random_set_counter = 0

for organism in eukaryotic_organisms:
    for motif in eukaryotic_motifs:
        EPDnew_promoter_positive_df = EPDnew_df.copy()[(EPDnew_df['organism'] == organism) &
                                                        (EPDnew_df['sigma factor/motif'] == motif) &
                                                        ((EPDnew_df['database/source'] == 'EPDnew'))]

        EPDnew_promoter_negative_permuted_df = EPDnew_df.copy()[(EPDnew_df['organism'] == organism) &
                                                                (EPDnew_df['sigma factor/motif'] == motif) &
                                                        (       (EPDnew_df['database/source'] == 'EPDnew (permuted)'))]
        EPDnew_promoters_for_ML_temp_df, random_set_counter = create_promoter_df_for_ml(EPDnew_promoter_positive_df,
                                      EPDnew_promoter_negative_permuted_df,
                                      random_df,
                                      percentage_permuted,
                                      random_set_counter)
        EPDnew_promoters_for_ML_df = pd.concat([EPDnew_promoters_for_ML_df,
           EPDnew_promoters_for_ML_temp_df],
            sort=False).reset_index().drop('index', axis=1)

In [ ]:
RegulonDB_promoters_for_ML_df = pd.DataFrame()

RegulonDB_promoter_positive_df = RegulonDB_df.copy()[RegulonDB_df['database/source'] == 'RegulonDB']
RegulonDB_promoter_negative_permuted_df = RegulonDB_df.copy()[RegulonDB_df['database/source'] == 'RegulonDB (permuted)']

RegulonDB_promoters_for_ML_df, random_set_counter = create_promoter_df_for_ml(RegulonDB_promoter_positive_df,
                              RegulonDB_promoter_negative_permuted_df,
                              random_df,
                              percentage_permuted,
                              random_set_counter)

In [ ]:
RegulonDB_promoters_for_ML_df